In [ ]:
import pickle
from datasets import Dataset, load_dataset
import openslide as ops
import os

In [ ]:
#dataset = load_dataset("Cilem/histopathology-1024")
dataset = pickle.load(open("./merged_embeddings/merged_dataset.pkl", "rb"))

In [ ]:
print(len(dataset))

In [ ]:
def search_slide(slide_path, slide_name):
    for root, dirs, files in os.walk(slide_path):
        for file in files:
            if slide_name in file:
                path = os.path.join(root, file)
                organ = path.split('/')[-2]
                return organ

In [ ]:
from collections import defaultdict

dset = []
organ_counts = defaultdict(int)
max_samples_per_organ = 14500

for i, d in enumerate(dataset):
    slide_name = d['slide_name']
    organ = search_slide('/home/cilem/Lfstorage/wsis/', slide_name)
    if organ_counts[organ] >= max_samples_per_organ:
        continue  # Belirtilen organ sayısına ulaşıldıysa atla
    
    d['organ'] = organ
    dset.append(d)
    organ_counts[organ] += 1



In [ ]:
print(len(dset))

from collections import Counter
print(Counter([d['organ'] for d in dset]))

In [ ]:
new_dset = []
for i, d in enumerate(dset):
    slide_name = d['slide_name']
    path = os.path.join('/home/cilem/Lfstorage/wsis', d['organ'], slide_name)
    x = d['x']
    y = d['y']
    patch_size = d['patch_size']
    slide = ops.OpenSlide(path)
    patch = slide.read_region((x, y), 0, patch_size)
    #patch = patch.resize((224, 224))
    d['patch_image'] = patch
    new_dset.append(d)

In [ ]:
new_dset = dset

In [ ]:
dataset = Dataset.from_dict({"organ": [item["organ"] for item in new_dset],
                             "slide_name": [item["slide_name"] for item in new_dset],
                             "x": [item["x"] for item in new_dset],
                             "y": [item["y"] for item in new_dset],
                             "level": [item["level"] for item in new_dset],
                             "patch_size": [item["patch_size"] for item in new_dset],
                             "resize": [item["resize"] for item in new_dset],
                             "image_embedding_vector": [item["embedding_vector"] for item in new_dset]})

In [ ]:
dataset.push_to_hub("Cilem/mixed-histopathology-512")